# Fit PRDX1 koffs and kons

This notebook performs analyses described in the 

**Manuscript:**

Modelling the decamerisation cycle of PRDX1 and the inhibition-like effect on its peroxidase activity

**Authors:**

C. Barry, C. Pillay, J. Rohwer

**Purpose:**

Writes Barranco-Medina 2008 Fig. 1c PRDX1 injection parameters (filename: `ITC_PRDX1_inject_params.csv`)

Performs regression on ITC data digitized from Barranco-Medina et al. 2008 to fit the kon and koff of Prx decamer formation.

Writes fitted koff and kon (filename: `fitted_koff_kon_PRDX1_params.csv`)


**Requirements:**

Python libraries (see **Imports** below)

ITC Prx model (filename:`ITC_Prx_dim-dec.psc`)

ITC model parameters (filename: `ITC_data_PRDX1_processed.csv`)

##### Imports

In [ ]:
import os
import math
import copy
import gc

import dill as pickle
import scipy as sp
import numpy as np
from lmfit import Minimizer, minimize, Parameters, Parameter, report_fit
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import auc
from scipy.interpolate import interp1d

import pysces


###### Save directories

In [ ]:
prev_dir = os.path.split(os.getcwd())[0]

mod_dir = os.path.join(prev_dir,"models")
if not os.path.isdir(mod_dir): os.mkdir(mod_dir) # ensure dir exists

par_dir = os.path.join(prev_dir,"params")
if not os.path.isdir(par_dir): os.mkdir(par_dir) # ensure dir exists
    
fig_dir = os.path.join(os.getcwd(),"Figures")
if not os.path.isdir(fig_dir): os.mkdir(fig_dir) # ensure dir exists


###### Matplotlib stuff

In [ ]:
%matplotlib inline

multiplier = 1
mpl_width = 4.5*multiplier
mpl_height = 3.5*multiplier
mpl_dpi = 600
mpl_xlabel_fontsize = "large"
mpl_ylabel_fontsize = "large"
mu = "\u03bc"

###### Fitting methods

In [ ]:
# fitting_method = "nelder"
fitting_method = "leastsq"

## Barranco-Medina et al. 2008 PRDX1 

### data

In [ ]:
df_PRDX1 = pd.read_csv(
    os.path.join(prev_dir,"ITC_data","ITC_data_PRDX1_processed.csv")
    ).set_index('Time (sec)')

### Inject params

In [ ]:
# Values from Barranco-Medina 2008 PRDX1
inj_enthalpy_PRDX1 = 156*5 # kcal/mol = ucal/nmol of decamer(x5)
inj_volume_PRDX1 = 1.6*10**(-3) # µl to ml
cell_volume_init = 1400*10**(-3) # µl to ml
inj_mon_PRDX1 = 102 # µM MONOMERS
inj_dim_est_PRDX1 = 0.8*inj_volume_PRDX1 # µM to nmol
inj_dec_est_PRDX1 = (inj_mon_PRDX1/2 - 0.8)/5*inj_volume_PRDX1 # µM to nmol
inj_interval_PRDX1 = 200 # s


In [ ]:
# Create df used to update models later
ITC_PRDX1_inject_params = {"Parameter": ["injection_enthalpy",
                                         "injection_volume",
                                         "injection_Prx_mon",
                                         "injection_dimer",
                                         "injection_decamer",
                                         "injection_interval",
                                         "first_inject_ratio",
                                         "cell_volume_init"],
         
                           "Value":[inj_enthalpy_PRDX1,
                                    inj_volume_PRDX1,
                                    inj_mon_PRDX1,
                                    inj_dim_est_PRDX1, 
                                    inj_dec_est_PRDX1,
                                    inj_interval_PRDX1,
                                    1,
                                    cell_volume_init]}

df_ITC_PRDX1_inject_params = pd.DataFrame(ITC_PRDX1_inject_params)
df_ITC_PRDX1_inject_params.set_index("Parameter",inplace=True)
df_ITC_PRDX1_inject_params


In [ ]:
ITC_PRDX1_inject_params = dict(zip(ITC_PRDX1_inject_params["Parameter"],ITC_PRDX1_inject_params["Value"]))

In [ ]:
# Write inject params to file
df_ITC_PRDX1_inject_params.to_csv(os.path.join(os.getcwd(),"ITC_PRDX1_inject_params.csv"),
                                  sep=',',
                                  encoding='utf-8')
df_ITC_PRDX1_inject_params.to_csv(os.path.join(par_dir, "ITC_PRDX1_inject_params.csv"),
                                  sep=',',
                                  encoding='utf-8')


## Fit functions

In [ ]:
def do_ITC_assay(mod, association_enthalpy, end = 3600):
    """
    Simulates and ITC trace and calculates the heat reading.
    Processes the heat reading data by correcting for baseline and normalizing to area under curve.
    """
    # Scan model
    mod.doSim(points=end+1, end=end)
    
    # Create pandas df from sim
    df_sim = pd.DataFrame(mod.sim)
    df_sim.rename(columns={'Time': 'Time (sec)'},inplace=True)
    df_sim['Time (sec)'] = np.rint(df_sim['Time (sec)']).astype(int)
    df_sim.set_index('Time (sec)',drop=True,inplace=True)
    
    # Calculate enthalpy
    df_sim["Rate μcal/sec"] = (-df_sim["Disassociation"])*association_enthalpy

    # Baseline correction
    df_sim = baseline_correction(df_sim, mod)
    
    df_sim.drop(0,inplace=True) 

    # Normalise to AUC
    df_sim = normalise_to_AUC(df_sim,mod)
    
    return df_sim

def baseline_correction(df_sim, mod):
    """
    Processes the heat reading data by correcting for baseline.
    """
    num_disso_injections = math.floor(df_sim["Rate μcal/sec"].index[-1]/mod.injection_interval)

    start = 1
    end = mod.injection_interval

    df_temp = pd.DataFrame(df_sim["Rate μcal/sec"].loc[start: end])

    next_baseline = df_temp[-10:-5]

    x2 = next_baseline.index.values.mean()
    y2 = next_baseline["Rate μcal/sec"].mean()

    x1 = 0.0
    y1 = next_baseline["Rate μcal/sec"].mean()

    baseline_interp = interp1d([x1,x2], [y1,y2],fill_value="extrapolate")(df_temp.index)

    current_baseline = next_baseline
    
    df_temp["Rate μcal/sec"] = df_temp["Rate μcal/sec"] - baseline_interp

    df_sim_baseline = pd.DataFrame(df_temp)

    count = 1
    while count < num_disso_injections:
        start = count*mod.injection_interval+1
        end = count*mod.injection_interval + mod.injection_interval

        df_temp = pd.DataFrame(df_sim["Rate μcal/sec"].loc[start: end])

        next_baseline = df_temp[-10:-5]

        x2 = next_baseline.index.values.mean()
        y2 = next_baseline["Rate μcal/sec"].mean()

        x1 = current_baseline.index.values.mean()
        y1 = current_baseline["Rate μcal/sec"].mean()

        baseline_interp = interp1d([x1,x2], [y1,y2],fill_value="extrapolate")(df_temp.index)

        df_temp["Rate μcal/sec"] = df_temp["Rate μcal/sec"] - baseline_interp

        current_baseline = next_baseline

        df_sim_baseline = pd.concat([df_sim_baseline,df_temp])

        count += 1

    df_sim["Rate μcal/sec no baseline corr."] = df_sim["Rate μcal/sec"].copy(deep=True)
    df_sim["Rate μcal/sec"] = df_sim_baseline["Rate μcal/sec"].copy(deep=True)

    return df_sim

def normalise_to_AUC(df_sim,mod):
    """
    Processes the heat reading data by normalizing for area under the curve.
    """
    num_disso_injections = math.floor(df_sim["Rate μcal/sec"].index[-1]/mod.injection_interval)

    start = 0
    end = mod.injection_interval
    df_temp = pd.DataFrame(df_sim["Rate μcal/sec"].loc[start: end])

    area = np.abs(np.trapz(df_temp.values.flatten(),df_temp.index))
    df_sim_AUCnorm = df_temp/area

    count = 1
    while count < num_disso_injections:
        start = count*mod.injection_interval
        end = start + mod.injection_interval
        df_temp = pd.DataFrame(df_sim["Rate μcal/sec"].loc[start: end])
        area = np.abs(np.trapz(df_temp.values.flatten(),df_temp.index))
        print(f"Area: {area}")
        df_temp = df_temp/area
        df_sim_AUCnorm = pd.concat([df_sim_AUCnorm,df_temp])
        count += 1

    df_sim_AUCnorm = df_sim_AUCnorm.reset_index().drop_duplicates(subset='Time (sec)', 
                                                                  keep='last').set_index('Time (sec)').sort_index()

    df_sim["Rate μcal/sec baseline no AUC"] = df_sim["Rate μcal/sec"].copy(deep=True)
    df_sim["Rate μcal/sec"] = df_sim_AUCnorm
    
    return df_sim

def update_syringe(mod_ITC,mod_syr,params):
    """
    Updates the ITC model with the injection dimers and decamers.
    """
    # Update model
    mod_syr.koff = params['koff'].value
    mod_syr.kon = params['koff'].value/params['Kd_app'].value
    mod_syr.exponent = params['exponent']

    # Find equilibrium
    mod_syr.doState()

    # Update ITC mod syringe params (dimers and decamers in injection in AMOUNT)
    setattr(mod_ITC, "injection_dimer", getattr(mod_syr,"dimers_ss")*getattr(mod_ITC,"injection_volume"))
    setattr(mod_ITC, "injection_decamer", getattr(mod_syr,"decamers_ss")*getattr(mod_ITC,"injection_volume"))

def do_first_inj(mod_ITC,mod_first_inj,params):
    """
    Updates the ITC model with the dimers and decamers of the first injection at equilibrium.
    """
    # Update model
    mod_first_inj.koff = params['koff'].value
    mod_first_inj.kon = params['koff'].value/params['Kd_app'].value
    mod_first_inj.exponent = params['exponent']

    # Find equilibrium
    mod_first_inj.doState()

    # Update ITC mod initial params (dimers and decamers in AMOUNT)
    setattr(mod_ITC, "dimers_init", getattr(mod_first_inj,"dimers_ss")*getattr(mod_ITC,"cell_volume_init"))
    setattr(mod_ITC, "decamers_init", getattr(mod_first_inj,"decamers_ss")*getattr(mod_ITC,"cell_volume_init"))

def fit_koff_kon_res(params, mod_ITC, mod_syr, mod_FI, inj_params, exp_data):
    """
    This is where the residuals which will be minimized are generated.
    Must contain the following:
    Update model params
    Scan model
    Calculate and return residuals
    """
    
    # Update ITC model
    mod_ITC.koff = params['koff'].value
    mod_ITC.kon = params['koff'].value/params['Kd_app'].value
    mod_ITC.exponent = params["exponent"]
    
    # Store kon in params
    params['kon'].value = params['koff'].value/params['Kd_app'].value
    
    # Update "syringe" dimers and decamers
    update_syringe(mod_ITC, mod_syr, params)
    
    # Update dimers and decamers per first injection
    mod_ITC.cell_volume = inj_params["cell_volume_init"]+inj_params["injection_volume"]
    do_first_inj(mod_ITC, mod_FI, params)
    
    # Scan model
    sim_data = do_ITC_assay(mod_ITC, inj_params["injection_enthalpy"])
    
    # Calculate residuals
    residuals = abs(exp_data["Rate μcal/sec"].values - sim_data.loc[exp_data.index]["Rate μcal/sec"].values)
    print(residuals)
    
    return residuals


## Load models

In [ ]:
# Assign injection parameters
inj_params = ITC_PRDX1_inject_params


In [ ]:
# ITC load model
mod_ITC_Prx_dim_dec = pysces.model("ITC_Prx_dim-dec.psc",dir = mod_dir)
mod_ITC_Prx_dim_dec.mode_integrate_all_odes = True # As recommented by pysces
mod_ITC_Prx_dim_dec.__settings__["cvode_access_solver"] = False
mod_ITC_Prx_dim_dec.__settings__["cvode_return_event_timepoints"] = False
mod_ITC_Prx_dim_dec.SetQuiet()
# Set injection params
for parameter in inj_params.keys():
    value = inj_params[parameter]
    setattr(mod_ITC_Prx_dim_dec,parameter,value)

# Load syringe model
mod_syringe = pysces.model("Prx_dim-dec.psc", dir = mod_dir)
mod_syringe.mode_integrate_all_odes = True # As recommented by pysces
mod_syringe.SetQuiet()
setattr(mod_syringe,
        "dimers_init", 
        inj_params["injection_dimer"]/inj_params["injection_volume"])
setattr(mod_syringe,
        "decamers_init", 
        inj_params["injection_decamer"]/inj_params["injection_volume"])

# Load first inject model
mod_first_inj = pysces.model("Prx_dim-dec.psc", dir = mod_dir)
mod_first_inj.mode_integrate_all_odes = True # As recommented by pysces
mod_first_inj.SetQuiet()
setattr(mod_first_inj, 
        "dimers_init", 
        inj_params["injection_dimer"]/(inj_params["injection_volume"]+inj_params["cell_volume_init"]))
setattr(mod_first_inj, 
        "decamers_init", 
        inj_params["injection_decamer"]/(inj_params["injection_volume"]+inj_params["cell_volume_init"]))



## Fit koff kon (figures 4a and 4b)

Fit the koff (and also kon) using the Barranco-Medina et al. 2008 PRDX1 data and ITC model.

In [ ]:
# Kd_app and exponent
exponent = 5
Kd_app_vil = 1.1 # Villar

In [ ]:
koff_min = 0.001
koff_max = 0.5

In [ ]:
# Copy models
mod_ITC = copy.deepcopy(mod_ITC_Prx_dim_dec)
mod_syr = copy.deepcopy(mod_syringe)
mod_FI = copy.deepcopy(mod_first_inj)

# Set up lmfit parameter library object with each parameter to be fitted
param_lib = Parameters()
param_lib.add('koff',value=0.01,min=koff_min,max=koff_max)
param_lib.add('kon',value=0.01,vary=False)
param_lib.add('Kd_app', value = Kd_app_vil,vary=False, min = 1e-12)
param_lib.add('exponent', value = exponent,vary=False, min = 1e-12)

# Perform fit
fit_koff_kon = minimize(fit_koff_kon_res, 
                         param_lib,
                         method=fitting_method,
                         args=(mod_ITC,
                               mod_syr,
                               mod_FI,
                               ITC_PRDX1_inject_params,
                               df_PRDX1),
                         epsfcn=0.0001)

In [ ]:
# Display fit report
fit_koff_kon 

In [ ]:
# Copy models
mod_ITC = copy.deepcopy(mod_ITC_Prx_dim_dec)
for param_name in fit_koff_kon.params.keys():
    setattr(mod_ITC,param_name,fit_koff_kon.params[param_name].value )
mod_syr = copy.deepcopy(mod_syringe)
mod_FI = copy.deepcopy(mod_first_inj)

# Update "syringe" dimers and decamers
update_syringe(mod_ITC, mod_syr, fit_koff_kon.params)

# Update dimers and decamers per first injection
mod_ITC.cell_volume = inj_params["cell_volume_init"]+inj_params["injection_volume"]
do_first_inj(mod_ITC, mod_FI, fit_koff_kon.params)
    
# Simulate ITC with fitted parameters
df_sim = do_ITC_assay(mod_ITC,ITC_PRDX1_inject_params["injection_enthalpy"])


In [ ]:
# Plot all fitted peaks
rows = 1
cols = 1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(df_sim.loc[df_PRDX1.index].index, 
           df_sim.loc[df_PRDX1.index]["Rate μcal/sec"],
           "b-", 
           label = "mod sim")
axarr.plot(df_PRDX1.index, df_PRDX1["Rate μcal/sec"],"k--", label = "exp data")

axarr.set_xlabel(f'Time (sec)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'{mu}cal/sec',fontsize = mpl_ylabel_fontsize)

f.tight_layout()
f.savefig(os.path.join(fig_dir,f"fit_koff_ITC_heat_vs_time.pdf"),dpi= mpl_dpi)

In [ ]:
# Plot single peak
inject_num = 2
injection_interval = 200

start = inject_num*injection_interval
end = start + injection_interval

rows = 1
cols = 1

f,axarr = plt.subplots(rows,cols)
f.set_size_inches(w=mpl_width*cols, h=mpl_height*rows)

axarr.plot(df_sim.loc[df_PRDX1.index].loc[start: end].index, 
           df_sim.loc[df_PRDX1.index].loc[start: end]["Rate μcal/sec"],
           "b-", 
           label = "mod sim")
axarr.plot(df_PRDX1.loc[start: end].index, 
           df_PRDX1.loc[start: end]["Rate μcal/sec"],
           "k--", 
           label = "exp data")

axarr.set_xlabel(f'Time (sec)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel(f'{mu}cal/sec',fontsize = mpl_ylabel_fontsize)

f.tight_layout()
f.savefig(os.path.join(fig_dir,f"fit_koff_ITC_heat_peak_vs_time.pdf"),dpi= mpl_dpi)
          

## Write fitted koff and kon to file

In [ ]:
# Set up df
data = {"Parameter": ["koff",
                     "kon",
                     "Kd_app"],
        "Value":[fit_koff_kon.params["koff"].value,
                fit_koff_kon.params["kon"].value,
                fit_koff_kon.params["Kd_app"].value]}
df_fitted_koff_kon_params = pd.DataFrame(data)
df_fitted_koff_kon_params.set_index("Parameter",inplace=True)
df_fitted_koff_kon_params

In [ ]:
# Write to file
df_fitted_koff_kon_params.to_csv(os.path.join(os.getcwd(),"fitted_koff_kon_PRDX1_params.csv"),
                                 sep=',',
                                 encoding='utf-8')
df_fitted_koff_kon_params.to_csv(os.path.join(par_dir,"fitted_koff_kon_PRDX1_params.csv"),
                                 sep=',',
                                 encoding='utf-8')
